In [13]:

setup = """
(deftemplate item
    (slot is)
    (slot id)
    (slot name)
    (slot priority)
)
"""
facts = []
with open("facts.txt", encoding="UTF-8") as f:
    for line in f:
        facts.append(line.rstrip())
with open("facts.clp", "w", encoding="UTF-8") as f:
    f.write(setup)
    f.write("(deffacts items \n")
    for fact in facts:
        s = fact.split(" # ")
        f.write(f"(item (is 0) (id {s[0]}) (name {s[1]}) (priority 1))\n")
    f.write(")")
    f.write("\n")


In [19]:

import re

setup = """

(deftemplate ioproxy ; шаблон факта-посредника для обмена информацией с GUI
	(slot message (default none))
)

(deffacts proxy-fact ; экземпляр факта ioproxy
	(ioproxy
		(message none)
	)
)

(defrule clear-message
	(declare (salience 90))
	?clear-msg-flg <- (clearmessage)
	?sendmessage <- (sendmessagehalt ?msg)
	=>
	(retract ?clear-msg-flg)
	(retract ?sendmessage)
)

(defrule set-output-and-halt
	(declare (salience 99))
	?current-message <- (sendmessagehalt ?new-msg)
	?proxy <- (ioproxy (message ?msg))
	=>
	(modify ?proxy (message ?new-msg))
	(retract ?current-message)
	(halt)
)
"""
rules = [{}]
with open("rules.txt", encoding="UTF-8") as f:
    for line in f:
        rule = re.search(r'(.*): (.*) -> (.*) #.* -> (.*)$', line)
        rules.append({"name": rule.group(1), "facts": rule.group(2), "out": rule.group(3), "description": rule.group(4)})
with open("rules.clp", "w", encoding="UTF-8") as f:
    f.write(setup)
    for rule in rules:
        if len(rule) == 0:
            continue
        f.write(f"(defrule {rule.get('name')} \n")
        f.write("\t(declare  (salience 50) ) \n")
        items = rule.get('facts').split(", ")
        for item in items:
            f.write(f"\t( item (id {item}) ) \n")
        f.write("\t=>\n")
        f.write(f"\t(assert ( item ( id {rule.get('out')}) ) )\n")
        f.write("\t(assert ( sendmessagehalt '{pep}')) \n".format(pep=rule.get('description')))
        f.write(")")
        f.write("\n")
        f.write("\n")
